## How to use

To enable selection (run a callback when you click on a rendered object) is enabled by two steps:

1) Run `scene.select(x,y)` when the user clicks on the canvas.
2) The scene will search the render_object that is drawn at the given pixel and triggers callbacks that are set via `render_object.on_select(callback)`. The callback gets one argument of type `webgpu.renderer.SelectEvent`.

The example below demonstrates this by drawing two types of objects (cones and cylinders) and registers two different callbacks. The selection is done via click with left mouse button (`ev['button'] == 0`). A string with selected type and number is shown on the top left of the canvas using a `Labels` object.

In [ ]:
import webgpu.jupyter as wj
import numpy as np
from webgpu.shapes import ShapeRenderer, generate_cylinder, generate_cone
from webgpu import Colormap, Labels, Colorbar

rand = np.random.random

thickness = 1e-2 * 10
length = 0.3
subdivision = 8
N = 10

# Render two types of objects: cones and cylinders
cylinder = generate_cylinder(subdivision, thickness)
cone = generate_cone(subdivision, thickness)

cmap_cone = Colormap(10, 11, "viridis")
cmap_cyl = Colormap(0, 1)
cbar_cone = Colorbar(cmap_cone)
cbar_cyl = Colorbar(cmap_cyl, (-0.9, 0.75))

cylinders = ShapeRenderer(
    cylinder, rand((N, 3)), length * rand((N, 3)), rand(2 * N), colormap=cmap_cyl
)
cones = ShapeRenderer(
    cone, rand((N, 3)), length * rand((N, 3)), 10.0 + rand(2 * N), colormap=cmap_cone
)

label = Labels(["", "Cones", "Cylinders"], [[-1, -1], [0.15, 0.9], [0.15, 0.75]], font_size=16)

scene = wj.Draw([cylinders, cones, label, cbar_cone, cbar_cyl])


def set_label(s):
    label.labels[0] = s
    label.set_needs_update()
    scene.render()


# Register callbacks for selection
def on_select_cone(ev):
    set_label(f"Selected cone {ev.uint32[0]} with value {ev.float32[1]}")


cones.on_select(on_select_cone)


def on_select_cyl(ev):
    set_label(f"Selected cylinder {ev.uint32[0]} with value {ev.float32[1]}")


cylinders.on_select(on_select_cyl)


def on_click(ev):
    if ev["button"] == 0:  # left mouse button
        # This finds the render object which is at the cursor and calls the 'on_select' callback of the correct render object
        scene.select(ev["canvasX"], ev["canvasY"])


# Call on_click when the user clicks on the canvas
scene.input_handler.on_click(on_click)

## How does it work?

Each render object that supports selection has two rendering pipelines, the "usual" one to render the object and a "selection_pipeline" to render information about the object encoded in the color of the resulting pixel. The selection pipeline has usually the same properties as the rendering pipeline, but a **different fragment shader**.

The code below shows the input struct and the two different fragment shaders for the ShapeRenderer above. Note that the output of a selection shader must have type `vec4<u32>`, i.e. four unsigned 32 bit integers. To store floats, you can use `bitcast<u32>(float_value)` and interpret the binary buffer accordingly on the python side.

```rust

// Input structure for fragment shaders
struct ShapeVertexOut {
    @builtin(position) position: vec4f,
    @location(0) p: vec3f,
    @location(1) normal: vec3f,
    @location(2) value: vec4f,
    @location(3) @interpolate(flat) instance: u32,
};

// Default fragment shader
@fragment fn shape_fragment_main_value(
    input: ShapeVertexOut,
) -> @location(0) vec4f {
    let color = getColor(input.value.x);
    return lightCalcColor(input.p, input.normal, color);
}

// Selection fragment shader
@fragment fn shape_fragment_main_select(
    input: ShapeVertexOut,
) -> @location(0) vec4<u32> {
    // @RENDER_OBJECT_ID@ has to be first here, that's used by the scene to know which render_object was drawn at this pixel
    // Write the instance (i.e. which cylinder is this) and the function value (must be casted to u32, well be casted back to float32 in python)
    return vec4<u32>(@RENDER_OBJECT_ID@, input.instance, bitcast<u32>(input.value.x), 0);
}

```


## Change the behavior

The example below creates a custom class `MyShapeRenderer` which is derived from `ShapeRenderer` and changes only the fragment shader code used for selection. Now we are storing the original position at the mouse cursor instead.

In [ ]:
select_shader = """
@fragment fn my_select_shader(
    input: ShapeVertexOut,
) -> @location(0) vec4<u32> {
    // Encode the current position int the last three values as unsigned integers
    return vec4<u32>(@RENDER_OBJECT_ID@, bitcast<vec3<u32>>(input.p));
}
"""


class MyShapeRenderer(ShapeRenderer):
    select_entry_point = "my_select_shader"

    def get_shader_code(self):
        return super().get_shader_code() + select_shader


mycylinders = MyShapeRenderer(cylinder, rand((N, 3)), rand((N, 3)), rand(2 * N))
mylabel = Labels([""], [[-1, -0.8]], font_size=14)


def on_select_mycyl(ev):
    mylabel.labels[0] = f"Selected mycylinder at point {ev.float32}"
    mylabel.set_needs_update()
    myscene.render()


mycylinders.on_select(on_select_mycyl)

myscene = wj.Draw([mycylinders, mylabel])

myscene.input_handler.on_click(lambda ev: myscene.select(ev["canvasX"], ev["canvasY"]))